In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 
import numpy as np
import pandas as pd # type: ignore
import random, os
import shutil
import matplotlib.pyplot as plt
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.metrics import categorical_accuracy 
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv('train.csv')

diagnosis_dict_binary = {
    0: 'No_DR',
    1: 'DR',
    2: 'DR',
    3: 'DR',
    4: 'DR'
}

diagnosis_dict = {
    0: 'No_DR',
    1: 'Mild',
    2: 'Moderate',
    3: 'Severe',
    4: 'Proliferate_DR',
}


df['binary_type'] =  df['diagnosis'].map(diagnosis_dict_binary.get) # type: ignore
df['type'] = df['diagnosis'].map(diagnosis_dict.get) # type: ignore
df.head(10)


,id_code,diagnosis,binary_type,type
0,000c1434d8d7,2,DR,Moderate
1,001639a390f0,4,DR,Proliferate_DR
2,0024cdab0c1e,1,DR,Mild
3,002c21358ce6,0,No_DR,No_DR
4,005b95c28852,0,No_DR,No_DR
5,0083ee8054ee,4,DR,Proliferate_DR
6,0097f532ac9f,0,No_DR,No_DR
7,00a8624548a9,2,DR,Moderate
8,00b74780d31d,2,DR,Moderate
9,00cb6555d108,1,DR,Mild


In [4]:
train, test = train_test_split(df, test_size = 0.2, stratify = df['binary_type'])

In [5]:
base_dir = ''

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)
os.makedirs(train_dir)

if os.path.exists(test_dir):
    shutil.rmtree(test_dir)
os.makedirs(test_dir)

In [7]:
src_dir = r'C:\Users\nanda\Documents\mini\aptos2019-blindness-detection\train_images'

for index,row in train.iterrows():
        binary_diagnosis = row['binary_type']
        id_code = row['id_code'] + ".png"
        srcfile = os.path.join(src_dir,id_code)
        dstfile = os.path.join(train_dir, binary_diagnosis)
        os.makedirs(dstfile, exist_ok = True)
        shutil.copy(srcfile, dstfile)

for index, row in test.iterrows():
        binary_diagnosis = row['binary_type']
        id_code = row['id_code'] + ".png"
        srcfile = os.path.join(src_dir, id_code)
        dstfile = os.path.join(test_dir, binary_diagnosis)
        os.makedirs(dstfile, exist_ok = True)
        shutil.copy(srcfile, dstfile)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\nanda\\Documents\\mini\\aptos2019-blindness-detection\\train_images\\799cb4c816ae.png'

In [7]:
train_batches = ImageDataGenerator(rescale = 1./255).flow_from_directory('train', target_size=(224,224),batch_size = 64, shuffle = True)
test_batches = ImageDataGenerator(rescale = 1./255).flow_from_directory('test', target_size=(224,224),batch_size = 64, shuffle = False)

Found 2929 images belonging to 2 classes.
Found 733 images belonging to 2 classes.


In [8]:
model = tf.keras.Sequential([ # type: ignore
    layers.Conv2D(8, (3,3), padding="valid", input_shape=(224,224,3), activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(16, (3,3), padding="valid", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(32, (4,4), padding="valid", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(64, (4,4), padding="valid", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
 
    layers.Flatten(),
    layers.Dense(64, activation = 'relu'),
    layers.Dropout(0.15),
    layers.Dense(2, activation = 'softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 1e-5), # type: ignore
              loss=tf.keras.losses.BinaryCrossentropy(), # type: ignore
              metrics=['accuracy'])

c:\Users\nanda\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
history = model.fit(train_batches,
                    epochs=15)

Epoch 1/15


c:\Users\nanda\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


46/46 ━━━━━━━━━━━━━━━━━━━━ 464s 8s/step - accuracy: 0.6957 - loss: 0.6341
Epoch 2/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 348s 6s/step - accuracy: 0.8528 - loss: 0.3416
Epoch 3/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 340s 6s/step - accuracy: 0.8690 - loss: 0.2964
Epoch 4/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 387s 7s/step - accuracy: 0.9055 - loss: 0.2596
Epoch 5/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 395s 7s/step - accuracy: 0.9123 - loss: 0.2362
Epoch 6/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 323s 6s/step - accuracy: 0.9160 - loss: 0.2249
Epoch 7/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 277s 5s/step - accuracy: 0.9237 - loss: 0.2033
Epoch 8/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 332s 6s/step - accuracy: 0.9301 - loss: 0.2060
Epoch 9/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 426s 8s/step - accuracy: 0.9316 - loss: 0.1979
Epoch 10/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 413s 8s/step - accuracy: 0.9277 - loss: 0.2146
Epoch 11/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 356s 7s/step - accuracy: 0.9383 - loss: 0.1868
Epoch 12/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 369s 7s/step - accuracy: 0.9302 - l

In [11]:

from tensorflow.keras import models
model.save('model.h5')  


In [12]:
loss     , acc = model.evaluate(test_batches, verbose=1)
print("Loss: ", loss)
print("Accuracy: ", acc)


12/12 ━━━━━━━━━━━━━━━━━━━━ 114s 9s/step - accuracy: 0.9255 - loss: 0.2108
Loss:  0.1970892995595932
Accuracy:  0.9345157146453857


In [13]:
import numpy as np
from tensorflow.keras.preprocessing import image 
test_image = image.load_img(r"C:\Users\nanda\Documents\mini_project\aptos2019-blindness-detection\testing_set\test_images\0e86fbc6cb8b.png", target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
if result[0][0] == 1:
  prediction = 'NO DR'
else:
  prediction = 'DR'

print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
NO DR
